In [152]:
import pandas as pd
import numpy as np

In [153]:
df_original = pd.read_csv("./raw.csv", index_col=False)
columns = [str(i) for i in  range(0, len(list(df_original)))]
df_original.columns = columns

In [154]:
df_original.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,1986,EK2986,C539,82623,C53.9,783313372,Konguta k�la,7,9,2015,...,NaN,NaN,NaN,9,Lõuna,Lõuna,9,diagnosed alive,33,1
1,449,EK1449,C539,89803,C53.9,377840000,Tallinn2018,7,4,2008,...,3,0,0,3,Harju,Tallinn,3,diagnosed alive,65,1
2,578,EK1578,C531,80003,C53.1,443220000,Kohtla-J�rve linn,0,9,2011,...,NaN,NaN,NaN,9,Ida-Viru,Ida-Viru,9,death certificate only,63,0
3,1600,EK2600,C538,80703,C53.8,863895766,Otsa k�la,7,9,2014,...,NaN,NaN,NaN,9,Lõuna,Lõuna,9,diagnosed alive,44,1
4,1329,EK2329,C539,80713,C53.9,706692020,Iira k�la,7,2,2015,...,NaN,NaN,NaN,9,Kesk,Kesk,9,diagnosed alive,52,1


In [155]:
#берем только dignosed alive

df = df_original[[
    '3',
    '9',
    '27',
    '17',
    '19',
    '18',
    '25',
    '28',
    '26',
]]
df = df.rename(columns={
    "3": "Code",
    "27": 'Dgnstat',
    "9": "Year", 
    "17": "Nationality",
    "18": "Education",
    "19": "Marital Status", 
    "28": "Age",
    "25": "Residence",
    "26": "Stage"
})

In [156]:
df.head()

,Code,Year,Dgnstat,Nationality,Marital Status,Education,Residence,Age,Stage
0,82623,2015,diagnosed alive,Estonian,married,University&higher,Lõuna,33,9
1,89803,2008,diagnosed alive,Estonian,widow,Secondary,Tallinn,65,3
2,80003,2011,death certificate only,Russian,divorced,Secondary,Ida-Viru,63,9
3,80703,2014,diagnosed alive,Estonian,divorced,Basic,Lõuna,44,9
4,80713,2015,diagnosed alive,Estonian,single,Secondary,Kesk,52,9


In [157]:
df = df.drop(df[df['Dgnstat'] != 'diagnosed alive'].index)

In [158]:
df['Education'] = df['Education'].replace(['Basic', 'Primary'], 'Basic&Primary')

In [159]:
df['Residence'] = df['Residence'].replace(['Tartu'], 'Lõuna')
df['Residence'] = df['Residence'].replace(['Tallinn'], 'Harju')
df['Residence'] = df['Residence'].replace(['Tartu'], 'Lõuna')
df['Residence'] = df['Residence'].replace(['Harju'], 'North')
df['Residence'] = df['Residence'].replace(['Lääne'], 'West')
df['Residence'] = df['Residence'].replace(['Kesk'], 'Central')
df['Residence'] = df['Residence'].replace(['Ida-Viru'], 'North-East')
df['Residence'] = df['Residence'].replace(['Lõuna'], 'South')

In [160]:
df.head()

,Code,Year,Dgnstat,Nationality,Marital Status,Education,Residence,Age,Stage
0,82623,2015,diagnosed alive,Estonian,married,University&higher,South,33,9
1,89803,2008,diagnosed alive,Estonian,widow,Secondary,North,65,3
3,80703,2014,diagnosed alive,Estonian,divorced,Basic&Primary,South,44,9
4,80713,2015,diagnosed alive,Estonian,single,Secondary,Central,52,9
5,80713,2013,diagnosed alive,Russian,divorced,Secondary,North-East,48,9


In [161]:
def groupCode(x) :
    x = int(x)
    if x >= 85603:
        g = 'AC'
    elif x >= 80500 and x <= 80789:
        g = 'SCC'
    elif x >= 80830 and x <= 80849:
        g = 'SCC'
    elif x >= 81400 and x <= 81419:
        g = 'AC'
    elif x >= 81900 and x <= 82119:
        g = 'AC'
    elif x >= 82300 and x <= 82319:
        g = 'AC'
    elif x >= 82600 and x <= 82659:
        g = 'AC'
    elif x >= 83100 and x <= 83109:
        g = 'AC'
    elif x >= 83800 and x <= 83809:
        g = 'AC'
    elif x >= 83820 and x <= 83849:
        g = 'AC'
    elif x >= 84400 and x <= 84909:
        g = 'AC'
    elif x >= 85700 and x <= 85749:
        g = 'AC'
    elif x >= 85760 and x <= 85769:
        g = 'AC'
    else:
        g = 'Unknown' 
    return g

def groupStage(x) :
    x = int(x)
    if x == 1:
        g = 0
    elif x >= 2 and x <= 4:
        g = 1
    else:
        g = 'Unknown'
    return g

def groupPeriod(x) :
    x = int(x)
    if x >= 2007 and x <= 2010:
        g = '2007-2010'
    elif x >= 2011 and x <= 2014:
        g = '2011-2014'
    elif x >= 2015 and x <= 2018:
        g = '2015-2018'
    else:
        g = 'Unknown'
    return g

def groupAge(x) :
    x = int(x)
    if x <= 29:
        g = '<= 29'
    elif x >= 30 and x <= 44:
        g = '30-44'
    elif x >= 45 and x <= 59:
        g = '45-59'
    elif x >= 60 and x <= 74:
        g = '60-74'
    elif x >= 75:
        g = '>= 75'
    return g

def groupNationality(x) :
    if x == 'Russian':
        x = 'Other'
    return x

In [162]:
df["CodeGroup"] = df['Code'].apply(groupCode)
df["StageGroup"] = df['Stage'].apply(groupStage)
df["Period"] = df['Year'].apply(groupPeriod)
df["AgeGroup"] = df['Age'].apply(groupAge)
df["Nationality"] = df['Nationality'].apply(groupNationality)
df = df.replace(['Unknown', 'unknown', 9], None)
df = df.drop("Dgnstat", axis=1)

In [163]:
df.head()

,Code,Year,Nationality,Marital Status,Education,Residence,Age,Stage,CodeGroup,StageGroup,Period,AgeGroup
0,82623,2015,Estonian,married,University&higher,South,33,None,AC,None,2015-2018,30-44
1,89803,2008,Estonian,widow,Secondary,North,65,3,AC,1,2007-2010,60-74
3,80703,2014,Estonian,divorced,Basic&Primary,South,44,None,SCC,None,2011-2014,30-44
4,80713,2015,Estonian,single,Secondary,Central,52,None,SCC,None,2015-2018,45-59
5,80713,2013,Other,divorced,Secondary,North-East,48,None,SCC,None,2011-2014,45-59


In [164]:
df.to_csv('prepared_data.csv', index=False)

In [165]:
df.Nationality.unique()

array(['Estonian', 'Other', None], dtype=object)

In [166]:
df.Education.unique()

array(['University&higher', 'Secondary', 'Basic&Primary',
       'Secondary_vocational', None], dtype=object)

In [167]:
df.Residence.unique()

array(['South', 'North', 'Central', 'North-East', 'West'], dtype=object)

In [168]:
df.Period.unique()

array(['2015-2018', '2007-2010', '2011-2014'], dtype=object)

In [169]:
df.AgeGroup.unique()

array(['30-44', '60-74', '45-59', '>= 75', '<= 29'], dtype=object)

In [170]:
df.Period.unique()

array(['2015-2018', '2007-2010', '2011-2014'], dtype=object)

In [171]:
df['CodeGroup'].unique()

array(['AC', 'SCC', None], dtype=object)

In [172]:
df['StageGroup'].unique()

array([None, 1, 0], dtype=object)